# 多人模式貪食蛇

剛剛我們一起做完貪食蛇遊戲了，這次我們大家連線一起玩貪食蛇吧！

### 遊戲規則

- 這次我們的貪食蛇是多人模式，每個人控制一條蛇，蛇的顏色不同
- 遊戲開始時，每條蛇都在地圖的隨機位置，並且有一個隨機的方向
- 每條蛇的目標是吃到地圖上的食物，吃到食物後蛇的身體會變長
- 當蛇的頭碰到地圖的邊界或者碰到任何蛇的身體時，蛇會死亡
- 你需要透過程式碼寫出你的蛇的移動策略，嘗試讓你的蛇吃到最多的食物！

### 連線方式

- 執行下面的程式碼，輸入你的名字，你就成功連線並開始和別的玩家比賽了！
- 然後[點我](https://youmingyeh.github.io/multiplayer-snake-game-websocket/)進入這個網址就可以看到你的蛇囉！

**_現在，請修改以下 `change_direction()` 函數，寫出屬於你的蛇的移動策略！_**


In [8]:
import asyncio
import socketio
import random
import json
sio = socketio.AsyncClient()
map = []
snakes = {}
foods = []
player_id = None
FRAME_RATE = 0.1

def direction_to_string(direction):
    if direction == [0, -1]:
        return "up"
    if direction == [0, 1]:
        return "down"
    if direction == [-1, 0]:
        return "left"
    if direction == [1, 0]:
        return "right"
    return None

@sio.event
async def connect():
    print("Connected to the server")
    await sio.emit("register", {"player_id": player_id})
    
@sio.event
async def disconnect():
    print("你死掉了！")
    
@sio.event
async def state(data):
    global map, snakes, foods, player_id
    data = json.loads(data)
    map = data["map"]
    snakes = data["snakes_pos"]
    foods = data["foods_pos"]

    if player_id is None:
        return

    if str(player_id) not in snakes.keys():
        await sio.emit("exit")
        await sio.disconnect()
        return
    
@sio.event
async def register(data):
    global player_id
    player_id = data["player_id"]
    print(f"成功註冊玩家！你的 ID 是： {player_id}")
    asyncio.create_task(send_moves())
    

def choose_direction(map, player_position, player_direction):
    ###########################################################
    # TODO: 修改你移動蛇的策略
    # 現在
    # direction: 你想要蛇的移動方向，可以是 "up", "down", "left", "right" 之一
    # player_position: 你的蛇的頭的位置
    # player_direction: 你的蛇的目前移動方向
    # map: 目前地圖上所有的資訊
    # map[i][j] = "" 代表地圖上(i, j)位置是空的
    # map[i][j] = "food" 代表地圖上(i, j)位置有食物
    # map[i][j] = "player" 代表地圖上(i, j)位置有其他玩家的蛇
    ###########################################################
    # 範例1: 隨機移動
    # directions = ["up", "down", "left", "right"]
    # direction = random.choice(directions)
    ###########################################################
    # 範例2: 不撞牆、不撞蛇移動
    # 請參考以下的程式碼
    directions = []
    if (
        player_position[0] > 0
        and (
            map[player_position[0] - 1][player_position[1]] == ""
            or map[player_position[0] - 1][player_position[1]] == "food"
        )
        and player_direction != "right"
    ):
        directions.append("left")
    if (
        player_position[0] < len(map) - 1
        and (
            map[player_position[0] + 1][player_position[1]] == ""
            or map[player_position[0] + 1][player_position[1]] == "food"
        )
        and player_direction != "left"
    ):
        directions.append("right")
    if (
        player_position[1] > 0
        and (
            map[player_position[0]][player_position[1] - 1] == ""
            or map[player_position[0]][player_position[1] - 1] == "food"
        )
        and player_direction != "down"
    ):
        directions.append("up")
    if (
        player_position[1] < len(map[0]) - 1
        and (
            map[player_position[0]][player_position[1] + 1] == ""
            or map[player_position[0]][player_position[1] + 1] == "food"
        )
        and player_direction != "up"
    ):
        directions.append("down")

    direction = random.choice(directions)
    return direction
    ###########################################################




async def send_moves():
    while True:
        global map, snakes, player_id, foods

        player_position = None
        player_direction = None
        if str(player_id) in snakes.keys():
            player_position = snakes[str(player_id)]["body"][0]
            player_direction = direction_to_string(snakes[str(player_id)]["direction"])

        if player_position is None or player_direction is None:
            await asyncio.sleep(FRAME_RATE)
            continue

        direction = choose_direction(map, player_position, player_direction)

        message = {"type": "move", "direction": direction}
        await sio.emit("move", message)
        await asyncio.sleep(FRAME_RATE)
async def main():
    global player_id
    player_id = input("輸入你的遊戲 ID: ")
    await sio.connect("https://ntuim-multiplayer-snake-game-server.azurewebsites.net/")
    await sio.wait()
import nest_asyncio
nest_asyncio.apply()
await main()

Connected to the server
成功註冊玩家！你的 ID 是： 葉又銘
